# ***IMPORTAR DATOS***

In [1]:
!mkdir data
!wget "https://drive.google.com/uc?export=download&id=1EG9_l6a5rRlTSzEwlNijqLuY7bxW_iuP" -O data/fulfilment_center_info.csv
!wget "https://drive.google.com/uc?export=download&id=19v-eC9LKjbAH0DMqQpi8Sb0XVxnNm9uq" -O data/meal_info.csv
!wget "https://drive.google.com/uc?export=download&id=1-zCbExblslglkus85ZCTNWwDh3TiEgkl" -O data/test.csv
!wget "https://drive.google.com/uc?export=download&id=1DZj2egxfzXYltmL_1M_atDHXMwz6IsR_" -O data/train.csv

--2020-04-13 21:39:48--  https://drive.google.com/uc?export=download&id=1EG9_l6a5rRlTSzEwlNijqLuY7bxW_iuP
Resolving drive.google.com (drive.google.com)... 108.177.126.138, 108.177.126.113, 108.177.126.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/07t30o57sapd2qo91jsca54neu7sv0c3/1586813925000/01100318546325464434/*/1EG9_l6a5rRlTSzEwlNijqLuY7bxW_iuP?e=download [following]
--2020-04-13 21:39:48--  https://doc-0c-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/07t30o57sapd2qo91jsca54neu7sv0c3/1586813925000/01100318546325464434/*/1EG9_l6a5rRlTSzEwlNijqLuY7bxW_iuP?e=download
Resolving doc-0c-00-docs.googleusercontent.com (doc-0c-00-docs.googleusercontent.com)... 108.177.126.132, 2a00:1450:4013:c01::84
Connecting to doc-0c-00-docs.googleusercontent.com (d

# ***Trabajo con los datos***

In [3]:
# Importamos las librerias 
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")
# Cargamos los datos 
fulfilment_center_info = pd.read_csv('data/fulfilment_center_info.csv')
meal_info = pd.read_csv('data/meal_info.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
dataset = pd.merge(train, fulfilment_center_info, how='inner', left_on='center_id', right_on='center_id')
dataset = pd.merge(dataset, meal_info, how='inner', left_on='meal_id', right_on='meal_id')
print(dataset.shape)
dataset.tail()
dataset.drop(['id'], axis=1, inplace=True)

dataset['checkout_price'] = np.log1p(dataset['checkout_price'])
dataset['base_price'] = np.log1p(dataset['base_price'])
dataset['num_orders'] = np.log1p(dataset['num_orders'])
dataset.head()

X = dataset.drop(['num_orders'], axis=1)
Y = dataset['num_orders']
X = X.drop(columns =["week","meal_id","center_id","city_code","region_code"], axis=1)
# Creamos los dummies del conjunto de datos
# 1. Convertimos todas las columnas a categóricas.
for n, col in dataset.items():
    if not pd.api.types.is_numeric_dtype(col) and n != 'num_orders':
        # Transforma la variable a tipo category
        dataset[n] = col.astype('category')
    
df = dataset.copy()

# 2. Convertir cada columna categórica a numérica.
for n,col in df.items():
    if pd.api.types.is_categorical_dtype(col):
        df[n] = col.cat.codes+1

# 3. Creamos los valores dummies.
X = pd.get_dummies(dataset.drop(['num_orders'], axis=1), drop_first=True)

X.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(456548, 15)


,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,op_area,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai
0,1,55,1885,4.926021,5.032332,0,0,647,56,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,55,1885,4.918739,5.032332,0,0,647,56,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,3,55,1885,4.897243,4.904682,0,0,647,56,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,55,1885,4.918959,4.911625,0,0,647,56,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,5,55,1885,4.993828,5.000585,0,0,647,56,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
X.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,center_type,op_area,category,cuisine
0,1,55,1885,136.83,152.29,0,0,647,56,TYPE_C,2.0,Beverages,Thai
1,2,55,1885,135.83,152.29,0,0,647,56,TYPE_C,2.0,Beverages,Thai
2,3,55,1885,132.92,133.92,0,0,647,56,TYPE_C,2.0,Beverages,Thai
3,4,55,1885,135.86,134.86,0,0,647,56,TYPE_C,2.0,Beverages,Thai
4,5,55,1885,146.50,147.50,0,0,647,56,TYPE_C,2.0,Beverages,Thai


In [6]:
Y.head

<bound method NDFrame.head of 0         177
1         323
2          96
3         163
4         215
         ... 
456543     13
456544     42
456545     40
456546     53
456547     27
Name: num_orders, Length: 456548, dtype: int64>

## **GridSearch**




In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()

parameters = [{'learning_rate': [0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]},
              {'n_estimators': [10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,120,140,160,180,200]},
              {'subsample': [0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]}]
clf = GridSearchCV(estimator=model,
                   param_grid=parameters,
                   scoring='accuracy')
clf.fit(X, Y)
accuracy=clf.best_score_
print(accuracy)
print(clf.best_params_)

ValueError: ignored

# ***XGBoost***

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X['center_type'] = labelencoder_X_1.fit_transform(X['center_type'])
labelencoder_X_2 = LabelEncoder()
X['category'] = labelencoder_X_2.fit_transform(X['category'])
labelencoder_X_2 = LabelEncoder()
X['cuisine'] = labelencoder_X_2.fit_transform(X['cuisine'])

onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(X).toarray()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.metrics import accuracy_score

y = Y
X = X

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = xgboost.XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(accuracy)

## ***PRUEBAS***

---

# **Pruebas**

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
kfolds = KFold(n_splits=5, shuffle=True, random_state=42)

y = Y
x = X

base_score, train_scores, val_scores, train_rmse, val_rmse = [], [], [], [], []
model = DecisionTreeRegressor(criterion='friedman_mse', max_depth=17)

for train_idx, val_idx in kfolds.split(x):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]
    model.fit(x_train, y_train)
    train_scores.append(model.score(x_train, y_train))
    val_scores.append(model.score(x_val, y_val))
    train_rmse.append(np.sqrt(mean_squared_error(y_train,model.predict(x_train))))
    val_rmse.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))
    y_val[:]=Y.mean()
    base_score.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))

print(f'Score Promedio')
print(f'Train      = {np.mean(train_scores):.4}')
print(f'Validation = {np.mean(val_scores):.4}')
print(f'RMSE Promedio')
print(f'Train      = {np.mean(train_rmse):.4}')
print(f'Validation = {np.mean(val_rmse):.4}')
print(f'Base Line Compare = {np.mean(base_score):.4}')

Score Promedio
Train      = 0.849
Validation = 0.7917
RMSE Promedio
Train      = 0.4676
Validation = 0.5491
Base Line Compare = 1.11


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
kfolds = KFold(n_splits=5, shuffle=True, random_state=42)

y = Y
x = X

base_score, train_scores, val_scores, train_rmse, val_rmse = [], [], [], [], []
model = RandomForestRegressor(n_estimators=10,criterion='friedman_mse',random_state = 10)

for train_idx, val_idx in kfolds.split(x):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]
    model.fit(x_train, y_train)
    train_scores.append(model.score(x_train, y_train))
    val_scores.append(model.score(x_val, y_val))
    train_rmse.append(np.sqrt(mean_squared_error(y_train,model.predict(x_train))))
    val_rmse.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))
    y_val[:]=Y.mean()
    base_score.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))
print(f'Score Promedio')
print(f'Train      = {np.mean(train_scores):.4}')
print(f'Validation = {np.mean(val_scores):.4}')
print(f'RMSE Promedio')
print(f'Train      = {np.mean(train_rmse):.4}')
print(f'Validation = {np.mean(val_rmse):.4}')
print(f'Base Line Compare = {np.mean(base_score):.4}')

Score Promedio
Train      = 0.9701
Validation = 0.8301
RMSE Promedio
Train      = 0.208
Validation = 0.496
Base Line Compare = 1.131


In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
kfolds = KFold(n_splits=5, shuffle=True)

y = Y
x = X

base_score, train_scores, val_scores, train_rmse, val_rmse = [], [], [], [], []

model = GradientBoostingRegressor(loss='lad',
                                n_estimators=100, max_depth=17,
                                learning_rate=0.5)

for train_idx, val_idx in kfolds.split(x):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]
    model.fit(x_train, y_train)
    train_scores.append(model.score(x_train, y_train))
    val_scores.append(model.score(x_val, y_val))
    train_rmse.append(np.sqrt(mean_squared_error(y_train,model.predict(x_train))))
    val_rmse.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))
    y_val[:]=Y.mean()
    base_score.append(np.sqrt(mean_squared_error(y_val,model.predict(x_val))))
print(f'Score Promedio')
print(f'Train      = {np.mean(train_scores):.4}')
print(f'Validation = {np.mean(val_scores):.4}')
print(f'RMSE Promedio')
print(f'Train      = {np.mean(train_rmse):.4}')
print(f'Validation = {np.mean(val_rmse):.4}')
print(f'Base Line Compare = {np.mean(base_score):.4}')